# SatelliteHub Quick Start

This notebook demonstrates the core features of SatelliteHub, a Python SDK for satellite data access and analysis. You'll learn how to:

1. **Create a location** - Define the geographic point you want to analyze
2. **Check vegetation health** - Get NDVI-based vegetation analysis
3. **Get weather data** - Retrieve temperature and precipitation
4. **Export results** - Save data to DataFrame, GeoTIFF, or PNG
5. **Explore raw data** - Access lower-level satellite data when needed

**Time to complete**: 10-15 minutes

**Prerequisites**: 
- SatelliteHub installed (`pip install satellitehub`)
- Copernicus credentials configured (for satellite data)
- CDS credentials configured (optional, for weather data)

## 1. Setup and Configuration

First, let's import SatelliteHub and check what's available.

In [ ]:
import satellitehub as sh

# Display the SDK version
print(f"SatelliteHub version: {sh.__version__}")

## 2. Create a Location

All analysis starts with a **location**. SatelliteHub uses WGS84 coordinates (the same as GPS).

Let's analyze a location in Poland - the Kampinos National Park near Warsaw, known for its forests and wetlands.

In [ ]:
# Create a location reference
# Kampinos National Park, Poland (52.3°N, 20.4°E)
field = sh.location(lat=52.30, lon=20.40)

# The location object captures the coordinates and determines the UTM zone
print(f"Location: {field}")
print(f"UTM Zone: {field.utm_zone}")
print(f"EPSG Code: EPSG:{field.utm_epsg}")

### What methods are available?

Use `available_methods()` to see what analysis you can run based on your configured credentials.

In [ ]:
# List available analysis methods
methods = field.available_methods()

print("Available methods:")
print("-" * 60)
for method in methods:
    print(method)

## 3. Vegetation Health Analysis

The `vegetation_health()` method computes the **Normalized Difference Vegetation Index (NDVI)** from Sentinel-2 satellite imagery.

**What is NDVI?**
- NDVI measures vegetation "greenness" using near-infrared and red light
- Values range from -1 to 1
- Healthy vegetation typically has NDVI > 0.3
- Values below 0.1 indicate bare soil, water, or stressed vegetation

**What happens behind the scenes:**
1. Query Copernicus Data Space for Sentinel-2 imagery
2. Filter for cloud-free scenes
3. Compute NDVI from red (B4) and near-infrared (B8) bands
4. Calculate quality metrics and confidence score

In [ ]:
# Analyze vegetation health for the last 30 days
# This queries Sentinel-2 imagery and computes NDVI
veg_result = field.vegetation_health(last_days=30)

# Display the result - SatelliteHub provides a narrative summary
print(veg_result)

### Understanding the Result

The `VegetationResult` object contains:

- **confidence**: How reliable the result is (0-1). Higher is better.
- **mean_ndvi**: Average vegetation index across the location
- **observations**: How many satellite passes were available
- **cloud_free_count**: How many passes had usable (cloud-free) data
- **warnings**: Any data quality issues

**Interpreting NDVI values:**
- `> 0.6` = Healthy, dense vegetation
- `0.3 - 0.6` = Moderate vegetation
- `0.1 - 0.3` = Sparse or stressed vegetation
- `< 0.1` = Bare soil, water, or no vegetation

In [ ]:
# Access individual properties
print(f"Mean NDVI: {veg_result.mean_ndvi:.3f}")
print(f"Confidence: {veg_result.confidence:.2f}")
print(f"Cloud-free observations: {veg_result.cloud_free_count} of {veg_result.observation_count}")

if veg_result.warnings:
    print("\nWarnings:")
    for warning in veg_result.warnings:
        print(f"  - {warning}")

## 4. Weather Data

Weather context helps interpret vegetation health. The `weather()` method retrieves temperature and precipitation data from ERA5 (global reanalysis) and optionally IMGW (Polish weather stations).

In [ ]:
# Get weather data for the same period
# Note: Requires CDS credentials. If unavailable, this will return with confidence=0
weather = field.weather(last_days=30)

print(weather)

In [ ]:
# Access weather metrics
if weather.confidence > 0:
    print(f"Mean temperature: {weather.mean_temperature:.1f}°C")
    print(f"Temperature range: {weather.temperature_min:.1f}°C to {weather.temperature_max:.1f}°C")
    print(f"Total precipitation: {weather.total_precipitation:.1f} mm")
    print(f"Data source: {weather.data_source}")
else:
    print("Weather data not available (CDS credentials may not be configured)")

## 5. Export Results

SatelliteHub results can be exported in multiple formats:

- **DataFrame**: For data analysis in pandas
- **GeoTIFF**: For GIS tools (raster data only)
- **PNG**: For presentations and reports

In [ ]:
# Export to pandas DataFrame
df = veg_result.to_dataframe()
print("Vegetation result as DataFrame:")
df

In [ ]:
# Export weather to DataFrame (if available)
if weather.confidence > 0:
    weather_df = weather.to_dataframe()
    print("Weather data as DataFrame:")
    print(weather_df.head())

In [ ]:
# Export to PNG visualization (uncomment to save)
# veg_result.to_png("vegetation_result.png")
# print("Saved vegetation visualization to vegetation_result.png")

## 6. Two-Tier API: Semantic vs Data

SatelliteHub offers two API levels:

### Semantic Tier (recommended for most use cases)
High-level methods like `vegetation_health()` that handle all the complexity:
- Automatic data acquisition
- Cloud masking
- Index computation
- Quality assessment

### Data Tier (for advanced users)
Low-level access via `field.data.*` for when you need:
- Specific bands or products
- Custom processing pipelines
- Raw data inspection

In [ ]:
# Data tier: Access raw Sentinel-2 data directly
# This gives you more control but requires more expertise
raw_result = field.data.sentinel2(
    bands=["B04", "B08"],  # Red and Near-Infrared bands
    last_days=30
)

print(f"Raw data result:")
print(f"  Confidence: {raw_result.confidence:.2f}")
print(f"  Data shape: {raw_result.data.shape if raw_result.data.size > 0 else 'empty'}")

## 7. Change Detection

Compare vegetation health between two time periods to detect changes.

**Use cases:**
- Drought impact assessment
- Deforestation monitoring
- Agricultural yield forecasting
- Post-fire recovery tracking

In [ ]:
# Compare vegetation from last month to the month before
change = field.vegetation_change(
    period_1_days=60,  # 30-60 days ago (baseline)
    period_2_days=30   # Last 30 days (comparison)
)

print(change)

In [ ]:
# Access change metrics
print(f"Period 1 NDVI: {change.period_1_ndvi:.3f}")
print(f"Period 2 NDVI: {change.period_2_ndvi:.3f}")
print(f"Change (delta): {change.delta:+.3f}")
print(f"Direction: {change.direction}")

## Summary

You've learned the core SatelliteHub workflow:

1. **Create location**: `sh.location(lat=..., lon=...)`
2. **Check availability**: `field.available_methods()`
3. **Analyze vegetation**: `field.vegetation_health(last_days=30)`
4. **Get weather**: `field.weather(last_days=30)`
5. **Detect changes**: `field.vegetation_change(...)`
6. **Export results**: `.to_dataframe()`, `.to_png()`, `.to_geotiff()`
7. **Access raw data**: `field.data.sentinel2(bands=[...])`

### Next Steps

- Explore the `consulting-template.ipynb` notebook for client-facing reports
- Check the API documentation for advanced options
- Try different locations across Poland

In [ ]:
# Quick test with your own location
# Replace with your coordinates:
# my_location = sh.location(lat=YOUR_LAT, lon=YOUR_LON)
# my_location.vegetation_health()